In [1]:
import pandas as pd

# Задание 1 #
Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:
- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений

In [2]:
vlog = pd.read_csv('visit_log.csv', sep=';')
vlog.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [12]:
%%time

# вариант с loc
vlog.loc[vlog.traffic_source.str.lower().isin(['yandex','google']), 'source_type'] = 'organic'
vlog.loc[(vlog.region == 'Russia') & (vlog.traffic_source == 'paid'), 'source_type'] = 'ad'
vlog.loc[~(vlog.region == 'Russia') & (vlog.traffic_source == 'paid'), 'source_type'] = 'other'
vlog.loc[pd.isnull(vlog.source_type), 'source_type'] = vlog.traffic_source

Wall time: 26 ms


In [23]:
%%time

# вариант с функцией, ради интереса - получилось намного медленнее
def src_t(row):
    if row['traffic_source'].lower() in ['yandex','google']:
        return 'organic'
    elif row['region'] == 'Russia' and row['traffic_source'] == 'paid':
        return 'ad'
    elif not row['region'] == 'Russia' and row['traffic_source'] == 'paid':
        return 'other'
    else:
        return row['traffic_source']

vlog['source_type'] = vlog.apply(src_t, axis=1)

Wall time: 640 ms


In [26]:
vlog.tail(10)

,timestamp,visit_id,url,region,user_id,traffic_source,source_type
18928,1550094255,c7e4bce62a,https://host.ru/8880fd00cf23066541,Australia,e721d23ec6,yandex,organic
18929,1550094260,39aeec73f8,https://host.ru/ee7c4e0674a284ee3f,Russia,9c832cf9d4,direct,direct
18930,1550094269,f1b4590410,https://host.ru/a5dda93e70318570c0,Russia,7f59a1e6a7,direct,direct
18931,1550094269,a7e2dfd759,https://host.ru/ef57543002ce2a0f2a,Belarus,04fec64cda,direct,direct
18932,1550094280,5001544772,https://host.ru/9f90e5c423a5f976d4,Russia,072148e3ba,google,organic
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,other
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,ad
18935,1550094308,e8cf2eb8e6,https://host.ru/a5dda93e70318570c0,Belarus,b85baa8c73,yandex,organic
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,ad
18937,1550094323,b3b634f824,https://host.ru/39fa884393666d45fc,Russia,7d27a58fb8,email,email


# Задание 2#
В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:
1. Прочитайте содержимое файла с датафрейм
2. Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном


In [33]:
urls = pd.read_csv('URLs.txt')
urls[urls.url.str.contains('\/\d{8}-', regex=True)]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


### Домашнее задание 3
В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [36]:
from pymystem3 import Mystem
data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})

In [41]:
%%time
m = Mystem()
data['lemmas'] = data['keyword'].apply(lambda kw: ' '.join(m.lemmatize(kw)))
data

Wall time: 2.83 s


,keyword,shows,lemmas
0,курс гривны к рублю,125076,курс гривна к рубль \n
1,доллары в рубли,114173,доллар в рубль \n
2,100 долларов в рублях,97534,100 доллар в рубль \n
3,курс рубля,53546,курс рубль \n
